# API Gateway

This book explores the API gateway using the AWS Python SDK

Relevant documentation:

* [Developer Guide](https://docs.aws.amazon.com/apigateway/latest/developerguide/welcome.html)
* [AWS Python SDK Docs](https://aws.amazon.com/sdk-for-python/)

In [ ]:
import boto3

apigw = boto3.client('apigateway')

## Endpoint Type

There are currently 3 endpoint types:

* Edge optimized
* Regional
* Private

## Create an API via Swagger Import

This imports a swagger spec, that points to an exiting endpoint.

In [ ]:
apiSpec = '''{
  "swagger": "2.0",
  "info": {
    "title": "Simple Pet Store via Swagger Import"
  },
  "schemes": [
    "https"
  ],
  "paths": {
    "/pets": {
      "get": {
        "responses": {
          "200": {
            "description": "200 response"
          }
        },
        "x-amazon-apigateway-integration": {
          "responses": {
            "default": {
              "statusCode": "200"
            }
          },
          "uri": "http://petstore-demo-endpoint.execute-api.com/petstore/pets",
          "passthroughBehavior": "when_no_match",
          "httpMethod": "GET",
          "type": "http"
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "type": "string"
          }
        ],
        "responses": {
          "200": {
            "description": "200 response"
          }
        },
        "x-amazon-apigateway-integration": {
          "responses": {
            "default": {
              "statusCode": "200"
            }
          },
          "requestParameters": {
            "integration.request.path.id": "method.request.path.petId"
          },
          "uri": "http://petstore-demo-endpoint.execute-api.com/petstore/pets/{id}",
          "passthroughBehavior": "when_no_match",
          "httpMethod": "GET",
          "type": "http"
        }
      }
    }
  }
}
'''

In [ ]:
response = apigw.import_rest_api(
    parameters={
        'endpointConfigurationTypes':'REGIONAL'
    },
    body=apiSpec
)

In [ ]:
print(response)

In [ ]:
restApiId = response['id']
print(restApiId)

In [ ]:
response = apigw.get_rest_api(
    restApiId=restApiId
)

In [ ]:
print(response)

### Clean Up

In [ ]:
response = apigw.delete_rest_api(
    restApiId=restApiId
)

In [ ]:
print(response)

## API Spec with Lambda Proxy Integration

### Create a Policy

In [ ]:
iamclient = boto3.client('iam')

In [ ]:
policyDoc='''{
    "Version": "2012-10-17",
    "Statement": [
      {
        "Effect": "Allow",
        "Action": [
          "logs:CreateLogStream",
          "logs:PutLogEvents",
          "logs:CreateLogGroup"
        ],
        "Resource": "arn:aws:logs:*:*:*"
      }
    ]
}'''

In [ ]:
response = iamclient.create_policy(
    PolicyName='minimal-lambda',
    Path='/',
    PolicyDocument=policyDoc,
    Description='minimal lambda policy'
)

In [ ]:
print(response)

In [ ]:
policyResponse = response # Update above to just use policy response
policyArn = policyResponse['Policy']['Arn']
print(policyArn)

### Create Role and Attach Policy

In [ ]:
assumeRolePolicyDocument='''{
    "Version": "2012-10-17",
    "Statement": [
      {
        "Effect": "Allow",
        "Principal": {
          "Service": [
            "lambda.amazonaws.com"
          ]
        },
        "Action": [
          "sts:AssumeRole"
        ]
      }
    ]
}'''

In [ ]:
roleResponse = iamclient.create_role(
    Path='/',
    RoleName='minimal-lambda-role',
    AssumeRolePolicyDocument=assumeRolePolicyDocument,
    Description='minimal lambda role'
)

In [ ]:
print(roleResponse)

In [ ]:
attachResponse = iamclient.attach_role_policy(
    RoleName=roleResponse['Role']['RoleName'],
    PolicyArn=policyArn
)

In [ ]:
print(attachResponse)

### Create the Lambda

In [ ]:
import random

s3client = boto3.client('s3')

# Bucket names are globally unique. Here we'll add a random element to the name to increase the odds of
# avoiding a name collision
bucket_name = 'ds-deploylambda-' + str(random.randint(1, 9999999))
print(bucket_name)

In [ ]:
response = s3client.create_bucket(
    Bucket=bucket_name
)

print(response)

In [ ]:
%%bash
ls
rm -rf code.zip
zip code.zip ./index.js

In [ ]:
s3 = boto3.resource('s3')
response = s3.meta.client.upload_file('./code.zip', bucket_name, 'code.zip')
print(response)

In [ ]:
lambdaClient = boto3.client('lambda')

In [ ]:
createLambdaFunctionResponse = lambdaClient.create_function(
    FunctionName='petStoreStub',
    Runtime='nodejs8.10',
    Role=roleResponse['Role']['Arn'],
    Handler='index.handler',
     Code={
        'S3Bucket': bucket_name,
        'S3Key': 'code.zip'
    }
);

print(createLambdaFunctionResponse)

### Create the API with Proxy Integration

First we need a role with the appropriate permissions to invoke the lambda from the gateway

In [ ]:
# TODO - inject the lambda arn as the resource - don't use * once it is working

policyDoc='''{
    "Version": "2012-10-17",
    "Statement": [
      {
        "Effect": "Allow",
        "Action": [
          "lambda:InvokeFunction"
        ],
        "Resource": "*"
      }
    ]
}'''

In [ ]:
lambdaPolicyResponse = iamclient.create_policy(
    PolicyName='generic-gateway-call-lambda',
    Path='/',
    PolicyDocument=policyDoc,
    Description='generic-gateway-call-lambda'
)

print(lambdaPolicyResponse)

In [ ]:
lambdaPolicyArn = lambdaPolicyResponse['Policy']['Arn']
print(lambdaPolicyArn)

In [ ]:
gatewayRolePolicyDoc = '''{
    "Version": "2012-10-17",
    "Statement": [
      {
        "Effect": "Allow",
        "Principal": {
          "Service": [
            "apigateway.amazonaws.com"
          ]
        },
        "Action": [
          "sts:AssumeRole"
        ]
      }
    ]
}'''

In [ ]:
gatewayRoleResponse = iamclient.create_role(
    Path='/',
    RoleName='gateway-lambda-role',
    AssumeRolePolicyDocument=gatewayRolePolicyDoc,
    Description='generic gateway role'
)

print(gatewayRoleResponse)

In [ ]:
attachResponse = iamclient.attach_role_policy(
    RoleName=gatewayRoleResponse['Role']['RoleName'],
    PolicyArn=lambdaPolicyArn
)

In [ ]:
print(attachResponse)

In [ ]:
apiSpec = '''{
  "swagger": "2.0",
  "info": {
    "title": "Simple proxy API"
  },
  "schemes": [
    "https"
  ],
  "paths": {
      "/{proxy+}": {
         "x-amazon-apigateway-any-method": {
            "parameters": [
               {
                  "name": "proxy",
                  "in": "path",
                  "required": true,
                  "schema": {
                     "type": "string"
                  }
               }
            ],
            "responses": {},
            "x-amazon-apigateway-integration": {
               "credentials":"'''
apiSpec = apiSpec + gatewayRoleResponse['Role']['Arn']
apiSpec = apiSpec + '''",
               "responses": {
                  "default": {
                     "statusCode": "200"
                  }
               },
               "uri": "arn:aws:apigateway:us-east-1:lambda:path/2015-03-31/functions/'''


In [ ]:
apiSpec = apiSpec + createLambdaFunctionResponse['FunctionArn'] 
apiSpec = apiSpec + '''/invocations",
               "passthroughBehavior": "when_no_match",
               "httpMethod": "POST",
               "cacheNamespace": "roq9wj",
               "cacheKeyParameters": [
                  "method.request.path.proxy"
               ],
               "type": "aws_proxy"
            }
         }
      }
   }
}
'''
print(apiSpec)


In [ ]:
response = apigw.import_rest_api(
    parameters={
        'endpointConfigurationTypes':'REGIONAL'
    },
    body=apiSpec
)
print(response)

In [ ]:
restApiId = response['id']
print(restApiId)

In [ ]:
response = apigw.get_rest_api(
    restApiId=restApiId
)

In [ ]:
print(response)

### Lambda with Custom Integration

In [ ]:
apiSpec = '''{
   "swagger":"2.0",
   "info":{
      "version":"2019-02-08",
      "title":"greeter"
   },
   "basePath":"/",
   "schemes":[
      "https"
   ],
   "paths":{
      "/hello":{
         "post":{
            "consumes":[
               "application/json"
            ],
            "produces":[
               "application/json"
            ],
            "parameters":[
               {
                  "in":"body",
                  "name":"Input",
                  "required":true,
                  "schema":{
                     "$ref":"#/definitions/Input"
                  }
               }
            ],
            "responses":{
               "200":{
                  "description":"200 response",
                  "schema":{
                     "$ref":"#/definitions/Result"
                  }
               }
            },
            "x-amazon-apigateway-request-validator": "Validate body",
            "x-amazon-apigateway-integration": {
               "credentials":"'''

apiSpec = apiSpec + gatewayRoleResponse['Role']['Arn']
apiSpec = apiSpec + '''",
               "responses": {
                  "default": {
                     "statusCode": "200"
                  }
               },
               "uri": "arn:aws:apigateway:us-east-1:lambda:path/2015-03-31/functions/'''

apiSpec = apiSpec + createLambdaFunctionResponse['FunctionArn'] 
apiSpec = apiSpec + '''/invocations",
              "passthroughBehavior":"when_no_templates",
              "httpMethod":"POST",
              "type":"aws"
            }
         }
      }
   },
   "definitions":{
      "Input":{
         "type":"object",
         "required":[
            "name",
            "salutation"
         ],
         "properties":{
            "name":{
               "type":"string"
            },
            "salutation":{
               "type":"string"
            }
         }
      },
      "Result":{
         "type":"object",
         "properties":{
            "greeting":{
               "type":"string"
            }
         }
      }
   },
   
  "x-amazon-apigateway-request-validators": {
    "Validate body": {
      "validateRequestParameters": false,
      "validateRequestBody": true
    }
  }
}'''

In [ ]:
print(apiSpec)

In [ ]:
response = apigw.import_rest_api(
    parameters={
        'endpointConfigurationTypes':'REGIONAL'
    },
    body=apiSpec
)
print(response)

In [ ]:
restApiId = response['id']
print(restApiId)

In [ ]:
response = apigw.get_rest_api(
    restApiId=restApiId
)

In [ ]:
print(response)

### Clean Up

In [ ]:
response = apigw.delete_rest_api(
    restApiId=restApiId
)
print(response)

In [ ]:
response = iamclient.detach_role_policy(
    RoleName=gatewayRoleResponse['Role']['RoleName'],
    PolicyArn=lambdaPolicyArn
)

print(response)

In [ ]:
response = iamclient.delete_role(
    RoleName=gatewayRoleResponse['Role']['RoleName']
)
print(response)

In [ ]:
response = iamclient.delete_policy(
    PolicyArn=lambdaPolicyArn
)
print(response)

In [ ]:
response = lambdaClient.delete_function(
    FunctionName='petStoreStub'
)

print(response)

In [ ]:
response = s3client.delete_object(
        Bucket=bucket_name,
        Key='code.zip'
)

print(response)

In [ ]:
response = s3client.delete_bucket(
    Bucket=bucket_name
)

print(response)

In [ ]:
response = iamclient.detach_role_policy(
    RoleName=roleResponse['Role']['RoleName'],
    PolicyArn=policyArn
)

print(response)

In [ ]:
response = iamclient.delete_role(
    RoleName=roleResponse['Role']['RoleName']
)
print(response)

In [ ]:
response = iamclient.delete_policy(
    PolicyArn=policyArn
)
print(response)